# AMQP et MQTT

<i>Si vous n'avez pas la possibilité d'éxécuter ce notebook (pas de .NET ou de Jupyter), vous pouvez utiliser [Binder](https://mybinder.org/v2/gh/mbaumanndev/prog-reseau-jupyter/master) pour l'éxécuter en ligne.</i>

AMQP et MQTT sont deux protocoles utilisés pour de la transmission de message. Bien que similaires, ils ont des cas d'utilisation différentes et des mécaniques qui divergent également un peu.

Fondamentatlement, ils utilisent une mécanique de Publish/Subscribe : Un programme (site web, service, ...) va pouvoir s'abonner à un ou plusieurs cannaux de communication en lecture ou en écriture afin de publier ou de recevoir des messages de façon asynchrone, à l'aide d'événements.

Habituellement, je fais étudier AMQP en cours. Avec le contexte actuel, nous allons plutôt étudier MQTT, car il y a de nombreuses ressources qui proposent des services MQTT à consommer en ligne. Ceci nous permettra de suivre le TP en même temps.

## Distinctions

MQTT va proposer de publier ou lire des messages dans des cannaux, que l'on va pouvoir distignuer par leurs chemins absolus. AMQP permet de faire la même chose, mais propose en plus des mécanismes de tag et d'autres routines pour dilivrer les messages à un ou plusieurs endroits différents avec un seul message émis.

## Utilisation dans le TP

Pour ce TP, nous allons utiliser une instance Mosquitto, un projet soutenu par Eclipse. Le serveur que nous allons utiliser est **public**, tout ce qui y sera échangé sera visible de tous.

## Avant de commencer

Avant d'entrer dans la partie programmation, nous allons installer une application sur smartphone afin de faire un test ensemble. Sur Android, vous pouvez installer [MQTT Dash](https://play.google.com/store/apps/details?id=net.routix.mqttdash&hl=fr), sur iOS, vous pouvez prendre [MQTTool](https://apps.apple.com/fr/app/mqttool/id1085976398).

### Instructions Android

Lancez MQTTDash, en haut à droite, vous trouverez un boutton "+", cliquez dessus et configurez les infos du serveurs de la façon suivante :

<img src="./images/amqp-mqtt/android-0.jpg" width="400" />

Une fois le serveur configuré, vous arrivez sur un nouvel écran : le dashboard. En haut à droite, vous trouverez à nouveau un bouton "+" permettant de configurer des widgets. Prennez un widget de type texte, et configurez-le de la façon suivante :

<img src="./images/amqp-mqtt/android-1.jpg" width="400" />

Vous avez maintenant un widget texte configuré et partagé avec le reste de la classe, dès que vous changez le texte du widget, toutes les personnes abonnées au topic le reçoive.

### Instructions iOS

Lancez MQTTool, et saisissez les informations du serveur comme ci-dessous :

<img src="./images/amqp-mqtt/iphone-0.jpg" width="400" />

Une fois que celà est fait, vous pouvez vous abonner à un topic de la façon suivante :

<img src="./images/amqp-mqtt/iphone-1.jpg" width="400" />

Vous pouvez de la même manière poster dans un topic depuis le troisième onglet. Notez que contrairement à MQTTDash sur Android, on n'a ici pas de dashboard, mais pour le cadre du cours ça suffira amplement.

## TP

Comme vu dans la configuration des applications mobiles, on va pouvoir s'abonner à un ou plusieurs topics. MQTT supporte différents types de wildcard, vous en trouverez des exemples [ici](https://github.com/chkr1011/MQTTnet/wiki/MQTT-topics). Ce lien GitHub est le dépôt de la librairie que nous allons utiliser aujourd'hui.

Comme pour la séance précédente, la cellule suivante va installer l'ensemble des dépendances dont nous aurons besoin et importer les namespaces nécessaires au fonctionnement du code (cette fois-ci, il y en aura peu).

Avant d'éxéuter la celulle suivante, assignez le contenu de la variable `v_Nom` à votre nom.

In [ ]:
#r "nuget:MQTTnet, 3.0.9"

using MQTTnet;
using MQTTnet.Client;
using MQTTnet.Client.Options;
using System.Threading;

var v_Nom = "Maxime"; // TODO: Mettez votre nom/pseudo ici
var v_ClientId = "lprgi2020-" + Guid.NewGuid();

Nous allons commencer par créer un simple client MQTT afin d'envoyer un message vers notre topic.

In [ ]:
var v_Factory = new MqttFactory();
var v_MqttClient = v_Factory.CreateMqttClient();

var v_Options = new MqttClientOptionsBuilder()
    .WithClientId(v_ClientId) // On a besoin d'un ID client unique
    .WithTcpServer("test.mosquitto.org", 1883)
    .Build();

await v_MqttClient.ConnectAsync(v_Options, CancellationToken.None);

Voilà, nous sommes connectés au serveur MQTT, maintenant, nous allons envoyer un message dans notre topic.

In [ ]:
var v_Message = new MqttApplicationMessageBuilder()
    .WithTopic("rgi2020")
    .WithPayload($"Hello {v_Nom}")
    .Build();

await v_MqttClient.PublishAsync(v_Message, CancellationToken.None);

Si tout se passe bien, vous devriez voir votre nom/pseudo saisi passer sur le dashboard ou dans la liste des messages, selon si vous êtes sous Android ou iOS.

Maintenant que nous savons envoyer des messages, nous allons attendre que des messages arrivent jusqu'à nous.

In [ ]:
v_MqttClient.UseConnectedHandler(async e =>
{
    await v_MqttClient.SubscribeAsync(new TopicFilterBuilder().WithTopic("rgi2020").Build());
});

Nous sommes maintenant abonnés au topic "rgi2020". Nous allons pouvoir regarder quels messages y passent... en théorie. En effet, les notebooks sont trop limités pour ce genre de tâches. Cependant, vous pouvez créer une application console avec l'ensemble du code qui a été écrit jusqu'à présent (avec un Main async) pour tester le bout de code suivant :

In [ ]:
v_MqttClient.UseApplicationMessageReceivedHandler(e =>
{
    Console.WriteLine($"Topic = {e.ApplicationMessage.Topic}");
    Console.WriteLine($"Payload = {Encoding.UTF8.GetString(e.ApplicationMessage.Payload)}");
    Console.WriteLine();
});

## À vous de jouer

Nous avons fait un tour rapide du protocole MQTT. Consultez à présent [les tutoriels de RabbitMQ](https://www.rabbitmq.com/getstarted.html) et essayez d'en faire le maximum. Il vous sera nécessaire d'installer RabbitMQ sur votre poste ou d'utiliser l'image Docker qui y est dédié. Je reste, bien entendu, disponible pour toutes questions pendant le créneau du cours si vous rencontrez un problème avec RabbitMQ.